# [NTDS'17] demo 6: web APIs & data analysis with pandas
[ntds'17]: https://github.com/mdeff/ntds_2017

[Michaël Defferrard](http://deff.ch), [EPFL LTS2](http://lts2.epfl.ch)

## 1 Web API

We already used the Twitter web API, albeit through a nice Python wrapper library. This time, we'll talk to an API directly. 

### 1.1 API doc

The first step is always to look at the API documentation. Two questions to answer: i) how to construct the uniform resource locator (URL), and ii) how to interpret the returned data. For this tutorial and the third assignment: <https://freemusicarchive.org/api>. (Some of you might have referred to the [Twitter API doc](https://developer.twitter.com/en/docs) for the first assignment.)

**Start simple**. Send an HTTP GET request with your web browser to the following uniform resource locator (URL): <https://freemusicarchive.org/recent.json>.

### 1.2 API calls

A well-designed [HTTP](https://en.wikipedia.org/wiki/Http) library for Python is [request](http://python-requests.org).

In [1]:
import requests

Send an HTTP GET request, as our browser did above, and receive a `response` from the web server.

In [2]:
URL = 'https://freemusicarchive.org/recent.json'
response = requests.get(URL)

If the GET request worked, the server answers with a "200 OK", the standard response for successful HTTP requests. The request may fail and e.g. return the infamous "404 Not Found" error.

In [3]:
print(response.status_code)
print(requests.get('https://www.epfl.ch/do_not_exist').status_code)

200
404


### 1.3 Exercise


First, as often, we need an API key for certain operations. Add the following to your `credentials.ini` file.
```
[freemusicarchive]
api_key = MY-KEY
```

In [4]:
# Read the confidential api key.
import configparser
import os
credentials = configparser.ConfigParser()
credentials.read(os.path.join('..', 'credentials.ini'))
api_key = credentials.get('freemusicarchive', 'api_key')

Find the name of the artist which has an ID of 58.

In [5]:
ARTIST_ID = 58

BASE_URL = 'https://freemusicarchive.org/api/get/artists.json'
url = '{}?artist_id={}&api_key={}'.format(BASE_URL, ARTIST_ID, api_key)
print(url)
requests.get(url).content

https://freemusicarchive.org/api/get/artists.json?artist_id=58&api_key=MN9IV2ZG9CJERJYQ


b'{"title":"Free Music Archive - Artists","message":"","errors":[],"total":"1","total_pages":1,"page":1,"limit":20,"dataset":[{"artist_id":"58","artist_handle":"Andy_Hayleck","artist_url":"http:\\/\\/freemusicarchive.org\\/music\\/Andy_Hayleck\\/","artist_name":"Andy Hayleck","artist_bio":"<p><span style=\\"font-family:Arial, Helvetica, sans-serif;\\">Electro-acoustic composer and instrumentalist Andy Hayleck                        is a quiet storm of the Baltimore experimental music scene,                        a virtuoso of sound whose range of highly developed projects                        and approaches is dizzying - spanning everything from jazz                        and ska guitar to musique concrete and the invention of                        new instruments and electro-acoustic systems. Hayleck \\"enjoy[s]                        collaborating with animate and inanimate objects.\\"                        In the animate realm, he has worked with free improvisors,             

## 2 JSON

The goal of an HTTP GET request is to get data. The returned data might be HTML (as you see when you browse the web), XML, JSON, etc. Most web APIs nowadays return data formated as [JSON](https://en.wikipedia.org/wiki/JSON). As JSON data objects consist of key-value pairs and lists, the format is well modeled by Python dictionaries and lists.

In [6]:
data = response.json()

The above call to `json()` interprets the returned data as being JSON and constructs Python dictionary and list objects out of it. In this case the top-level object is a dictionary, with some keys.

In [7]:
print(type(data))
print(data.keys())

<class 'dict'>
dict_keys(['content', 'aFeeds', 'title', 'page_title', 'page_cache_key', 'oNav', 'nav_genres', 'nav_curators', 'bAuth', 'bManage', 'sSearch', 'aMailForm', 'aTracks'])


Let's look at the value of the "title" key.

In [8]:
data['title']

'Free Music Archive'

Exploring the returned data is a good way to learn about the API. Let's get to what we were looking for, a list of recently added tracks.

In [9]:
print(type(data['aTracks']))
print(data['aTracks'][0].keys())

<class 'list'>
dict_keys(['track_id', 'album_id', 'artist_id', 'agreement_id', 'track_title', 'track_handle', 'language_code', 'track_image_file', 'track_duration', 'track_number', 'track_disc_number', 'track_explicit', 'track_explicit_notes', 'track_upload_confirmed', 'track_file', 'track_file_type', 'track_bit_rate', 'track_sample_rate', 'track_encoding_software', 'track_original_id3', 'track_copyright_c', 'track_copyright_p', 'track_composer', 'track_lyricist', 'track_publisher', 'track_instrumental', 'track_information', 'track_notes', 'track_status_encoding', 'track_status', 'track_date_recorded', 'track_date_created', 'track_date_modified', 'track_date_deleted', 'track_date_published', 'track_comments', 'track_favorites', 'track_listens', 'track_downloads', 'track_interest', 'track_deleted', 'artist_name', 'album_title', 'license_id', 'license_parent_id', 'license_title', 'license_url', 'license_image_file', 'license_image_file_large', 'language_title', 'artist_published', 'artis

In [10]:
for track in data['aTracks'][:5]:
    print(track['track_title'])

explore deming
Nmap
#!!!!@{[€].............................--º«...............
Black Hat Python
Host: 192.168.0.1 () Ports: 139/open/tcp//netbios-ssn///


### 2.1 Exercise

Construct a list of the names of the 16 top-level genres. No need to call the API again, everything is in the above collected JSON data.

In [11]:
genres = [genre['genre_title'] for genre in data['nav_genres']]

assert type(genres) is list
print(genres)

['Blues', 'Classical', 'Country', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Novelty', 'Old-Time / Historic', 'Pop', 'Rock', 'Soul-RnB', 'Spoken']


## 3 Pandas: data analysis in Python

While it might be sufficient to keep the data as lists and dictionaries, we often prefer to see data in a tabular format for analysis. A tabular format allows to make operations on the rows and columns, e.g. by taking a sum over prices. At a large scale, tabular data is stored in a [database](https://en.wikipedia.org/wiki/Database) (think the list of clients of a compagny). At a small scale, you probably used it in the form of an Excel spreadsheet. In Python, [pandas](https://pandas.pydata.org) is the most used data analysis tool. You can think of it as a *programmable spreadsheet*.

Let's first create a simple table, called a `DataFrame` in pandas' language. We can initialize the table with e.g. a Python list or a NumPy array. As our running example, let's say we want to do some accounting for our family and define the following schema: each row is a member of the family, the first column represents the revenue and the second the expenses. Sure enough, we can create a NumPy array.

In [12]:
import numpy as np

accounts = np.array([[10, 20], [30, 30], [40, 20]])
print(accounts)

[[10 20]
 [30 30]
 [40 20]]


But that's not very user friendly. Who's the second line already? Enter pandas.

In [13]:
import pandas as pd

accounts = pd.DataFrame(accounts)
accounts

,0,1
0,10,20
1,30,30
2,40,20


But this is not much more useful than our NumPy array. Let's e.g. name the rows and columns.

In [14]:
accounts.columns = ['revenues', 'expenditures']
accounts.index = ['John', 'Mary', 'Alison']
accounts.index.name = 'given name'
accounts

,revenues,expenditures
given name,,
John,10,20
Mary,30,30
Alison,40,20


Now if I want to know how much Alison spent this month, I don't have to remember that Alison is the third row and that the expenditures are the second column. I can query:

In [15]:
accounts.at['Alison', 'expenditures']

20

We may want to compute the revenue of the entire familiy (note the similarity with the way you would do it in a spreadsheet):

In [16]:
accounts['revenues'].sum()

80

Or the balance of each member:

In [17]:
accounts['balance'] = accounts['revenues'] - accounts['expenditures']
accounts

,revenues,expenditures,balance
given name,,,
John,10,20,-10
Mary,30,30,0
Alison,40,20,20


Another quite useful feature is selection:

In [18]:
accounts[accounts['balance'] < 0]

,revenues,expenditures,balance
given name,,,
John,10,20,-10


Or sorting:

In [19]:
accounts.sort_values('expenditures')

,revenues,expenditures,balance
given name,,,
John,10,20,-10
Alison,40,20,20
Mary,30,30,0


Now it's time to save our data for archival, or to open it up in another tool.

In [20]:
accounts.to_csv(os.path.join('..', 'data', 'family_accounts.csv'))

In [21]:
!cat ../data/family_accounts.csv
# Windows: !type ..\data\family_accounts.csv

given name,revenues,expenditures,balance
John,10,20,-10
Mary,30,30,0
Alison,40,20,20


These are very basic operations to give you an idea of what pandas is. More info in the [docs](http://pandas.pydata.org/pandas-docs/stable/). That library will certainly be useful for your projects.

### 3.1 Exercise

Using pandas and the above data (i.e. `data['aTracks']`), find how many tracks each artist published.

In [22]:
tracks = pd.DataFrame(data['aTracks'])
assert type(tracks) is pd.DataFrame
tracks.head()

,agreement_id,album_id,album_title,album_url,artist_id,artist_name,artist_published,artist_url,language_code,language_title,...,track_number,track_original_id3,track_publisher,track_sample_rate,track_status,track_status_encoding,track_title,track_upload_confirmed,track_url,track_user_favorite
0,29393,24719,explore deming,http://freemusicarchive.org/music/The_Anchorit...,25291,The Anchorites,1,http://freemusicarchive.org/music/The_Anchorites/,None,None,...,1,"{""attached or linked picture"":{""description"":""...",None,44100,published,complete,explore deming,1,http://freemusicarchive.org/music/The_Anchorit...,0
1,29392,24718,Error,http://freemusicarchive.org/music/crk/Error_1916/,14023,.crk,1,http://freemusicarchive.org/music/crk/,None,None,...,5,"{""album"":""Error"",""artist"":"".CrK"",""genre"":""Brea...",None,44100,published,complete,Nmap,1,http://freemusicarchive.org/music/crk/Error_19...,0
2,29392,24718,Error,http://freemusicarchive.org/music/crk/Error_1916/,14023,.crk,1,http://freemusicarchive.org/music/crk/,None,None,...,4,"{""album"":""Error"",""artist"":"".CrK"",""genre"":""Brea...",None,44100,published,complete,#!!!!@{[€].............................--º«......,1,http://freemusicarchive.org/music/crk/Error_19...,0
3,29392,24718,Error,http://freemusicarchive.org/music/crk/Error_1916/,14023,.crk,1,http://freemusicarchive.org/music/crk/,None,None,...,3,"{""album"":""Error"",""artist"":"".CrK"",""genre"":""Brea...",None,44100,published,complete,Black Hat Python,1,http://freemusicarchive.org/music/crk/Error_19...,0
4,29392,24718,Error,http://freemusicarchive.org/music/crk/Error_1916/,14023,.crk,1,http://freemusicarchive.org/music/crk/,None,None,...,2,"{""album"":""Error"",""artist"":"".CrK"",""genre"":""Brea...",None,44100,published,complete,Host: 192.168.0.1 () Ports: 139/open/tcp//netb...,1,http://freemusicarchive.org/music/crk/Error_19...,0


In [23]:
tracks['artist_name'].value_counts()

Mike Durek        8
.crk              5
Scott Holmes      4
Lobo Loco         2
The Anchorites    1
Name: artist_name, dtype: int64